In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.style.use("seaborn-white")
import seaborn as sns
sns.set_style("white")
from torch import nn
from tqdm import tqdm
from toolz import compose
import datetime
import numpy as np
import pandas as pd
import torch
from torch.utils import data
import logging
import random
import os

In [2]:
from data import (
    test_images_path,
    load_images_as_arrays,
    TGSSaltDataset,
    prepare_test_data,
)
from model import model_path, predict_tta
from data import rle_encode
from resnetlike import UNetResNet
from config import load_config

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
now = datetime.datetime.now()

In [5]:
config = load_config()["EvaluateModel"]
logger.info(f"Loading config {config}")

INFO:config:Loading config ../configs/config.json
INFO:__main__:Loading config {'base_channels': 32, 'batch_size': 128, 'id': '61464e07-ce3e-42b1-b88a-482d645756d8', 'img_target_size': 101, 'initial_model_filename': 'best_lovasz_model.pth', 'name': 'EvaluateModel', 'num_workers': 0, 'seed': 42, 'threshold': 0.32277339226305085}


In [6]:
locals().update(config)

In [7]:
torch.backends.cudnn.benchmark = True
logger.info(f"Started {now}")

INFO:__main__:Started 2018-10-05 12:59:04.783109


In [8]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [9]:
model = UNetResNet(1, base_channels)

In [10]:
device = torch.device("cuda:0")
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): UNetResNet(
    (enc1): EncodingLayer(
      (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (res1): PreactivationResidualBlock(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (res2): PreactivationResidualBlock(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

In [11]:
model.module.final_activation = nn.Sequential().to(device)

In [12]:
model_dir = os.path.join(model_path(), f"{id}")
filename = os.path.join(model_dir, initial_model_filename)
checkpoint = torch.load(filename)
model.load_state_dict(checkpoint["state_dict"])

In [13]:
test_df = prepare_test_data()
x_test = np.array(load_images_as_arrays(test_df.index, test_images_path()))
x_test = x_test.reshape(-1, 1, img_target_size, img_target_size)

100%|██████████| 18000/18000 [00:14<00:00, 1223.11it/s]


In [14]:
dataset_test = TGSSaltDataset(x_test, is_test=True)

In [15]:
test_data_loader = data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False,
)

In [16]:
model.eval()
predictions = [predict_tta(model, image) for image in tqdm(test_data_loader)]

100%|██████████| 141/141 [00:33<00:00,  5.02it/s]


In [17]:
preds_test = np.concatenate(predictions, axis=0).squeeze()

In [18]:
transform = compose(rle_encode, np.round, lambda x: x > threshold)

In [19]:
pred_dict = {
    idx: transform(preds_test[i]) for i, idx in enumerate(tqdm(test_df.index.values))
}

100%|██████████| 18000/18000 [00:05<00:00, 3055.75it/s]


In [20]:
sub = pd.DataFrame.from_dict(pred_dict, orient="index")
sub.index.names = ["id"]
sub.columns = ["rle_mask"]
filename = os.path.join(model_dir, f"submission_{now:%d%b%Y_%H}.csv")
sub.to_csv(filename)